# Test alpha-spheres network 2

In [1]:
import molsysmt as msm

In [2]:
from scipy.spatial import Voronoi
from math import sqrt

In [3]:
from scipy.spatial.distance import euclidean

In [4]:
import numpy as np

In [5]:
from tqdm import tqdm

In [6]:
from test import is_permeable_2, is_permeable_3, is_permeable_4

In [7]:
molecular_system = msm.convert('1HIV')
molecular_system = msm.extract(molecular_system, selection='molecule_type in ["protein", "peptide"]')

In [8]:
msm.info(molecular_system)

form,n_atoms,n_groups,n_components,n_chains,n_molecules,n_entities,n_proteins,n_structures
molsysmt.MolSys,1516,198,2,2,2,1,2,1


## Castp

In [9]:
pocket = []
with open('castp/1hiv/1hiv.poc') as file:
    for line in file:
        fields = line.rstrip().split()
        if fields[-2]=='1':
            pocket.append(int(fields[1]))

pocket = msm.get(molecular_system, element='atom', selection='atom_id==@pocket', atom_index=True)

In [10]:
view=msm.view(molecular_system)
view

NGLWidget()

In [11]:
view.clear()
#view.add_surface(color='w', opacity=0.5)
view.add_ball_and_stick(color='w', opacity=0.5)

In [12]:
sel='@'+','.join([str(ii) for ii in pocket])
#view.add_surface(sel, color='red', opacity=0.3)
view.add_ball_and_stick(sel, color='red', opacity=0.3)

In [13]:
pocket

array([  66,   67,   68,   69,   70,   71,  180,  181,  196,  197,  201,
        206,  207,  208,  210,  211,  213,  214,  218,  219,  220,  221,
        222,  225,  226,  229,  243,  365,  367,  368,  369,  370,  371,
        372,  373,  374,  375,  376,  377,  378,  381,  382,  383,  401,
        402,  403,  585,  610,  616,  617,  618,  624,  625,  640,  641,
        661,  825,  826,  827,  828,  829,  830,  939,  940,  956,  960,
        965,  966,  967,  969,  970,  972,  973,  977,  978,  979,  980,
        981,  984,  985,  988, 1002, 1121, 1124, 1126, 1127, 1128, 1129,
       1130, 1131, 1132, 1133, 1136, 1140, 1141, 1143, 1161, 1162, 1370,
       1376, 1377, 1383, 1399, 1400, 1420])

# Mine

In [14]:
radio = {'C': 1.7, 'H':1.1, 'O':1.52, 'N':1.55, 'P':1.8, 'S':1.8}

In [15]:
coordinates = msm.get(molecular_system, coordinates=True)

In [16]:
coordinates = msm.pyunitwizard.get_value(coordinates[0])
n_atoms = coordinates.shape[0]

In [17]:
atom_types = msm.get(molecular_system, element='atom', atom_type=True)
radii = np.array([radio[ii]*0.1 for ii in atom_types])
probe = 0.1

In [18]:
voronoi = Voronoi(coordinates)

In [19]:
centers = voronoi.vertices
n_alphaspheres = centers.shape[0]

In [20]:
atoms_of_alphasphere = {ii:[] for ii in range(n_alphaspheres)}
atoms_of_alphasphere[-1] = []
alphaspheres_of_atom = {ii:[] for ii in range(n_atoms)}

In [21]:
for atom_index, region_index in enumerate(voronoi.point_region):
    for alphasphere_index in voronoi.regions[region_index]:
        atoms_of_alphasphere[alphasphere_index].append(atom_index)
        alphaspheres_of_atom[atom_index].append(alphasphere_index)

In [22]:
alphaspheres_in_castp = []
for ii in range(n_alphaspheres):
    if np.isin(atoms_of_alphasphere[ii], pocket).all():
        alphaspheres_in_castp.append(ii)

In [23]:
#alphaspheres_in_castp

In [24]:
alphaspheres_in_castp[-1]

9548

In [25]:
atoms_of_alphasphere[9548]

[610, 617, 618, 1140]

In [26]:
pocket

array([  66,   67,   68,   69,   70,   71,  180,  181,  196,  197,  201,
        206,  207,  208,  210,  211,  213,  214,  218,  219,  220,  221,
        222,  225,  226,  229,  243,  365,  367,  368,  369,  370,  371,
        372,  373,  374,  375,  376,  377,  378,  381,  382,  383,  401,
        402,  403,  585,  610,  616,  617,  618,  624,  625,  640,  641,
        661,  825,  826,  827,  828,  829,  830,  939,  940,  956,  960,
        965,  966,  967,  969,  970,  972,  973,  977,  978,  979,  980,
        981,  984,  985,  988, 1002, 1121, 1124, 1126, 1127, 1128, 1129,
       1130, 1131, 1132, 1133, 1136, 1140, 1141, 1143, 1161, 1162, 1370,
       1376, 1377, 1383, 1399, 1400, 1420])

In [27]:
import networkx as nx

rp = 0.3

G = nx.Graph()

for ii in tqdm(range(-1,n_alphaspheres)):

    G.add_node(ii)
    
    neighbors=[]
    for jj in atoms_of_alphasphere[ii]:
        for kk in alphaspheres_of_atom[jj]:
            if kk>ii:
                neighbors.append(kk)
    neighbors=np.unique(neighbors)
    
    for jj in neighbors:
        common = np.intersect1d(atoms_of_alphasphere[ii], atoms_of_alphasphere[jj], assume_unique=True)
        if len(common)==2:
            discs = coordinates[common]
            rs = radii[common]
            if is_permeable_2(rp, discs, rs):
                G.add_edge(ii,jj,face=common)            
        elif len(common)==3:
            discs = coordinates[common]
            rs = radii[common]
            if is_permeable_3(rp, discs, rs):
                G.add_edge(ii,jj,face=common)
        elif len(common)==4:
            discs = coordinates[common]
            rs = radii[common]
            if is_permeable_4(rp, discs, rs):
                G.add_edge(ii,jj,face=common)
        elif len(common)>4:
            print('ups')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9811/9811 [00:07<00:00, 1300.19it/s]


In [28]:
G.number_of_edges()

2181

In [29]:
G.number_of_nodes()

9811

In [30]:
components = nx.connected_components(G)

In [31]:
veo = list(components)

In [32]:
len(veo)

9150

In [33]:
len(veo[0])

653

In [ ]:
asphere_A = alphaspheres_in_castp[0]
asphere_B = alphaspheres_in_castp[2]

asphere_A = 8007
asphere_B =8008

face = G[asphere_A][asphere_B]['face']
face_coordinates = coordinates[face]

In [ ]:
face_coordinates.mean(axis=1)

In [ ]:
centroid = face_coordinates.mean(axis=1)

In [ ]:
euclidean(centroid, face_coordinates[0])

In [ ]:
euclidean(centroid, face_coordinates[1])

In [ ]:
euclidean(centroid, face_coordinates[2])

In [ ]:
view=msm.view(molecular_system)

view.clear()
#view.add_surface(color='w', opacity=0.5)
view.add_ball_and_stick(color='w', opacity=0.5)

for ii in veo[0]:
    if ii in G[-1]:
        aa=atoms_of_alphasphere[ii]
        view.shape.add_cylinder(coordinates[aa[0]]*10, coordinates[aa[1]]*10, [1,0,0], 0.1)
        view.shape.add_cylinder(coordinates[aa[0]]*10, coordinates[aa[2]]*10, [1,0,0], 0.1)
        view.shape.add_cylinder(coordinates[aa[0]]*10, coordinates[aa[3]]*10, [1,0,0], 0.1)
        view.shape.add_cylinder(coordinates[aa[1]]*10, coordinates[aa[2]]*10, [1,0,0], 0.1)
        view.shape.add_cylinder(coordinates[aa[1]]*10, coordinates[aa[3]]*10, [1,0,0], 0.1)
        view.shape.add_cylinder(coordinates[aa[2]]*10, coordinates[aa[3]]*10, [1,0,0], 0.1)

view